In [1]:
import os
import cv2
import numpy as np

# Paths to your images and masks
image_dir = '../dataset2/images/'  # Replace with your images folder path
mask_dir = '../dataset2/masks/'    # Replace with your masks folder path

# Output directories for augmented images and masks
output_image_dir = '../dataset2/augmented_images/'
output_mask_dir = '../dataset2/augmented_masks/'

# Create output directories if they don't exist
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# Function to flip and save images and masks
def flip_and_save(image, mask, name_prefix, image_output_dir, mask_output_dir):
    # Original
    cv2.imwrite(os.path.join(image_output_dir, f"{name_prefix}_original.png"), image)
    cv2.imwrite(os.path.join(mask_output_dir, f"{name_prefix}_original.png"), mask)
    
    # Horizontally flipped
    h_flip_image = cv2.flip(image, 1)  # Flip code 1 = horizontal flip
    h_flip_mask = cv2.flip(mask, 1)
    cv2.imwrite(os.path.join(image_output_dir, f"{name_prefix}_hflip.png"), h_flip_image)
    cv2.imwrite(os.path.join(mask_output_dir, f"{name_prefix}_hflip.png"), h_flip_mask)

    # Vertically flipped
    v_flip_image = cv2.flip(image, 0)  # Flip code 0 = vertical flip
    v_flip_mask = cv2.flip(mask, 0)
    cv2.imwrite(os.path.join(image_output_dir, f"{name_prefix}_vflip.png"), v_flip_image)
    cv2.imwrite(os.path.join(mask_output_dir, f"{name_prefix}_vflip.png"), v_flip_mask)

    # Horizontally and vertically flipped
    hv_flip_image = cv2.flip(image, -1)  # Flip code -1 = horizontal and vertical flip
    hv_flip_mask = cv2.flip(mask, -1)
    cv2.imwrite(os.path.join(image_output_dir, f"{name_prefix}_hvflip.png"), hv_flip_image)
    cv2.imwrite(os.path.join(mask_output_dir, f"{name_prefix}_hvflip.png"), hv_flip_mask)

# Loop through images and masks
for file_name in os.listdir(image_dir):
    if file_name.endswith('.png') or file_name.endswith('.jpg'):  # Process only image files
        # Load the image and its corresponding mask
        image_path = os.path.join(image_dir, file_name)
        mask_path = os.path.join(mask_dir, file_name)
        
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path)
        
        if image is None or mask is None:
            print(f"Error loading {file_name}, skipping...")
            continue
        
        # Ensure the mask matches the image dimensions
        if image.shape != mask.shape:
            print(f"Dimension mismatch for {file_name}, skipping...")
            continue
        
        # Use the base name (without extension) for saving augmented files
        name_prefix = os.path.splitext(file_name)[0]
        
        # Apply flips and save augmented images and masks
        flip_and_save(image, mask, name_prefix, output_image_dir, output_mask_dir)

print("Augmentation completed. Augmented images and masks saved to the specified folders.")

Augmentation completed. Augmented images and masks saved to the specified folders.


In [6]:
import os
from PIL import Image, ImageStat
import numpy as np

def augment_masks_and_images(mask_dir, image_dir, output_mask_dir, output_image_dir, green_threshold=1):
    """
    Augments mask and image pairs by applying mirroring and rotation transformations.

    Args:
        mask_dir (str): Path to the directory containing input mask images.
        image_dir (str): Path to the directory containing input images.
        output_mask_dir (str): Path to the directory where augmented masks will be stored.
        output_image_dir (str): Path to the directory where augmented images will be stored.
        green_threshold (float): Maximum allowed percentage of green component in masks to qualify for augmentation.

    Returns:
        None
    """
    if not os.path.exists(output_mask_dir):
        os.makedirs(output_mask_dir)

    if not os.path.exists(output_image_dir):
        os.makedirs(output_image_dir)

    for filename in os.listdir(mask_dir):
        mask_path = os.path.join(mask_dir, filename)
        image_path = os.path.join(image_dir, filename)

        try:
            with Image.open(mask_path) as mask, Image.open(image_path) as img:
                mask = mask.convert("RGB")  # Ensure the mask is in RGB mode
                img = img.convert("RGB")    # Ensure the image is in RGB mode

                # Check the green component percentage in the mask
                mask_pixels = np.array(mask)
                total_pixels = mask_pixels.shape[0] * mask_pixels.shape[1]
                green_pixels = np.sum(mask_pixels[:, :, 1] > mask_pixels[:, :, 0])  # Green > Red
                green_pixels += np.sum(mask_pixels[:, :, 1] > mask_pixels[:, :, 2])  # Green > Blue
                green_percentage = green_pixels / (2 * total_pixels)

                if green_percentage <= green_threshold:
                    # Perform augmentations
                    transformations = {
                        "vertical_mirror": (mask.transpose(Image.FLIP_TOP_BOTTOM), img.transpose(Image.FLIP_TOP_BOTTOM)),
                        "horizontal_mirror": (mask.transpose(Image.FLIP_LEFT_RIGHT), img.transpose(Image.FLIP_LEFT_RIGHT)),
                        "rotate_90_left": (mask.rotate(90, expand=True), img.rotate(90, expand=True)),
                        "rotate_90_right": (mask.rotate(-90, expand=True), img.rotate(-90, expand=True)),
                        "rotate_180": (mask.rotate(180, expand=True), img.rotate(180, expand=True)),
                        "original":(mask,img)
                    }

                    for suffix, (transformed_mask, transformed_img) in transformations.items():
                        mask_output_filename = f"{os.path.splitext(filename)[0]}_{suffix}.png"
                        image_output_filename = f"{os.path.splitext(filename)[0]}_{suffix}.png"

                        mask_output_path = os.path.join(output_mask_dir, mask_output_filename)
                        image_output_path = os.path.join(output_image_dir, image_output_filename)

                        transformed_mask.save(mask_output_path)
                        transformed_img.save(image_output_path)
                else:
                    mask.save(mask_output_path)
                    img.save(image_output_path)
        except Exception as e:
            print(f"Skipping file {filename} due to error: {e}")

# Example usage:
augment_masks_and_images("../dataset2/masks/", "../dataset2/images/", "../dataset2/augmented_masks_v2", "../dataset2/augmented_images_v2")
print("DONE")

DONE
